# Training Pipeline

# Imports
## Pip Packages

In [1]:
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

if not os.path.exists('data'):
    new_directory_path = "..\\..\\"
    os.chdir(new_directory_path)

from src.datasets import FreeViewInMemory, seq2seq_jagged_collate_fn, seq2seq_padded_collate_fn
from src.model import PathModel
import numpy as np

# Dataset

## Load Dataset

In [2]:
# TODO Implement Strategy and Builder pattern
datasetv2 = FreeViewInMemory(sample_size= 13,log = True, start_index=2)


Data loaded in memory


# Training

In [3]:
model = PathModel(input_dim = 3,
                  output_dim = 3,
                  n_encoder = 2,
                  n_decoder = 2,
                  model_dim = 256,
                  total_dim = 256,
                  n_heads = 4,
                  ff_dim = 512,
                  max_pos_enc=15,
                  max_pos_dec=26)

In [4]:


dataloader = DataLoader(datasetv2, batch_size=128, shuffle=True, num_workers=0, collate_fn= seq2seq_padded_collate_fn)
for batch in tqdm(dataloader):
    x,x_mask,y, y_mask, fixation_len = batch
    break
    

  0%|          | 0/274 [00:02<?, ?it/s]


In [9]:
print(y_mask.shape)

torch.Size([128, 14])


In [7]:
# model = torch.compile(model)
reg_out,cls_out = model(x,y)

In [8]:
reg_out.shape

torch.Size([128, 14, 3])

In [ ]:
print(reg_out.shape)
print(cls_out.shape)

torch.Size([128, 14, 3])
torch.Size([128, 14, 1])


In [ ]:
def compute_loss(reg_out,cls_out, y, padding_mask, fixation_len, cls_weigth = .5):
    criterion_reg = torch.nn.MSELoss()
    criterion_cls = torch.nn.BCEWithLogitsLoss()
    y_mask = torch.logical_not(padding_mask)
    cls_targets = torch.zeros(cls_out.size(), dtype = torch.float32)
    batch_idx = torch.arange(cls_targets.size()[0])
    cls_targets[batch_idx,fixation_len] = True
    cls_loss = criterion_cls(cls_out[y_mask], cls_targets[y_mask])
    # test masking: print for every item in the batch the last non masked value to check if they are one
    for i in range(cls_targets.size()[0]):
        print(cls_targets[i, y_mask[i]].long()[-1])
    # Regression loss

    # Classification loss
    # reg_loss = criterion_reg(reg_out[y_mask], y[y_mask])
     # Example target: 1 if point exists, else 0

    # total_loss = (1-cls_weigth)*reg_loss + cls_weigth*cls_loss
    # return total_loss

compute_loss(reg_out,cls_out, y, y_mask, fixation_len)

IndexError: The shape of the mask [128, 13] at index 1 does not match the shape of the indexed tensor [128, 14, 1] at index 1

In [ ]:
fixation_len

tensor([ 8,  7,  7,  7,  5, 10,  9,  7, 10,  7,  7,  7, 13,  9,  8,  5,  6,  7,
         6,  9,  8, 10,  8,  7,  8,  5,  6,  9,  8,  5, 10,  7,  7,  7,  8,  8,
         6,  8, 12, 12,  4,  6, 10,  6,  6,  7,  8,  7,  6,  8,  6,  4,  7,  9,
         4,  4, 11,  7,  4,  7,  9,  7,  9, 13,  6,  9,  6,  7,  9,  8,  5,  6,
         8,  7,  5,  7,  4,  5,  7,  6,  6,  4,  8,  7,  6,  8,  8, 10,  9,  8,
         5,  5,  5,  6,  9, 12,  6,  6,  9,  7,  9,  9,  7, 13,  9,  9, 11, 11,
        11,  6,  9,  9,  7,  7,  8,  6,  7,  8,  6, 10,  8,  4, 10,  6,  7,  8,
         5,  7])